# Installation

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [1]:
#%pip install onnx
#%pip install onnxruntime
#%pip install onnxruntime-training
%pip install ultralytics
!pip install roboflow
!pip install torch
!pip install comet_ml
!pip install clearml


  Using cached comet_ml-3.39.2-py3-none-any.whl (638 kB)
  Using cached everett-3.1.0-py2.py3-none-any.whl (35 kB)
  Using cached python_box-6.1.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (3.3 MB)
  Using cached semantic_version-2.10.0-py2.py3-none-any.whl (15 kB)
  Using cached sentry_sdk-1.44.0-py2.py3-none-any.whl (264 kB)
  Using cached simplejson-3.19.2-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl (137 kB)
  Using cached websocket_client-1.3.3-py3-none-any.whl (54 kB)
  Using cached wurlitzer-3.0.3-py3-none-any.whl (7.3 kB)
  Using cached dulwich-0.21.7-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (514 kB)
  Using cached configobj-5.0.8-py2.py3-none-any.whl (36 kB)
  Attempting uninstall: websocket-client
    Found existing installation: websocket-client 1.7.0
    Uninstalling websocket-client-1.7.0:
      Successfully uninstalled websocket-client-1.7.0
  Attempting uninstall: python-box
    Found

In [2]:
import comet_ml
comet_ml.init()
#experiment = comet_ml.Experiment()

In [ ]:
import clearml
clearml.browser_login()

Access key : 08608ISYLPL31P1TY66I

Secret key : PiOwo6qZYIGCl6PQrMEVUqWXNaDicAW6fga0gC14SuVlFcGswA

# Git Clone

In [ ]:
#pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="4RVq4o5W9hxfxscEyTFv")
project = rf.workspace("natthakorn").project("new_helmetdetection")
version = project.version(5)
dataset = version.download("yolov8")

In [ ]:
%cd /content
!git clone https://github.com/ultralytics/ultralytics.git

In [ ]:
import torch
import subprocess
def check_gpu_usage():
    process = subprocess.Popen(["nvidia-smi"], stdout=subprocess.PIPE)
    output, err = process.communicate()
    return output

print(check_gpu_usage())
torch.cuda.empty_cache()
print(check_gpu_usage())



# Train



In [ ]:
from ultralytics import YOLO
import subprocess
import torch
import os
import shutil

torch.cuda.set_per_process_memory_fraction(0.87)
torch.cuda.empty_cache()

def check_gpu_usage():
    process = subprocess.Popen(["nvidia-smi"], stdout=subprocess.PIPE)
    output, err = process.communicate()
    return output

print(check_gpu_usage())

#model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

#parameter
myoptimizer = 'RAdam' #SGD, Adam, AdamW, NAdam, RAdam, RMSProp

path = "/content/New_HelmetDetection-5/data.yaml"
results = model.train(data=path, time=1.5, batch=-1 ,
                      imgsz=640, save_period = 20,cos_lr=True,optimizer=myoptimizer)
#saveperiod = 5 for size=1920 and 20 for size=640 #cos_lr=True

torch.cuda.empty_cache()

results = model.val()

success = model.export(format='onnx')

source_path = '/content/runs/detect'
destination_path = '/content/drive/MyDrive/Yolov8_Project/_Weight_/2024-03-30/wn'
shutil.copytree(source_path, destination_path)

b'Sat Mar 30 09:56:30 2024       \n+---------------------------------------------------------------------------------------+\n| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |\n|-----------------------------------------+----------------------+----------------------+\n| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |\n| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |\n|                                         |                      |               MIG M. |\n|=========================================+======================+======================|\n|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |\n| N/A   42C    P0              26W /  70W |    103MiB / 15360MiB |      2%      Default |\n|                                         |                      |                  N/A |\n+-----------------------------------------+------------

100%|██████████| 6.23M/6.23M [00:00<00:00, 73.1MB/s]


Ultralytics YOLOv8.1.37 🚀 Python-3.10.12 torch-2.2.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/content/New_HelmetDetection-5/data.yaml, epochs=100, time=1.5, patience=100, batch=-1, imgsz=640, save=True, save_period=20, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=RAdam, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=True, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_co

100%|██████████| 755k/755k [00:00<00:00, 16.7MB/s]


Overriding model.yaml nc=80 with nc=4

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

...

Closing dataloader mosaic
albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      84/93      10.6G     0.7784     0.5332      1.023        115        640: 100%|██████████| 35/35 [00:52<00:00,  1.49s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:03<00:00,  1.84s/it]

                   all        245        749      0.789      0.714      0.799      0.567



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      85/93      10.6G     0.7487     0.5123      1.006        172        640:  14%|█▍        | 5/35 [00:03<00:25,  1.17it/s]

In [ ]:
experiment.end()

In [ ]:
from ultralytics import YOLO

model = YOLO('yolov8n.yaml')
model = YOLO('yolov8n.pt')

path = '/content/Helmet_01-3/data.yaml'
results = model.train(data=path, epochs=100, batch=-1 , imgsz=640, save_period = 20, lr0=0.001) #0.001

results = model.val()

# Prediction

In [ ]:
!pip install ultralytics
!pip install onnxruntime
!pip install onnx

In [ ]:
from ultralytics import YOLO

pathv2 = '/content/drive/MyDrive/1_Yolov8_Pjct/_Weight_/weights/best.onnx'
model = YOLO(pathv2)

model.predict('/content/drive/MyDrive/1_Yolov8_Pjct/ImagesForTest',
              save=True,
              conf=0.5)

# Train ต่อจากเดิม

**1**

In [ ]:
#Train ต่อจากของเดิม

from ultralytics import YOLO
import subprocess
import torch
import os
import shutil

def check_gpu_usage():
    process = subprocess.Popen(["nvidia-smi"], stdout=subprocess.PIPE)
    output, err = process.communicate()
    return output

print(check_gpu_usage())


path_pretrain = '/content/runs/detect/train/weights/best.pt'
model = YOLO(path_pretrain)

torch.cuda.empty_cache()
torch.cuda.set_per_process_memory_fraction(0.85)

path = '/content/New_HelmetDetection-5/data.yaml'
results = model.train(data=path, time=2, batch=-1, imgsz=640, save=True, save_period = 20,
                      cos_lr=True,plots=True,verbose=True)

torch.cuda.empty_cache()

results = model.val()

success = model.export(format='onnx')

source_path = '/content/runs/detect/train2'
destination_path = '/content/drive/MyDrive/Yolov8_Project/_Weight_/2024-03-30/train2'
shutil.copytree(source_path, destination_path)

# ย้ายไฟล์ไปยัง Google Drive

In [ ]:
import shutil
# ค้นหาเส้นทางของโฟลเดอร์ต้นทาง
source_path = '/content/runs/detect'

# สร้างเส้นทางของโฟลเดอร์ปลายทาง
destination_path = '/content/drive/MyDrive/Yolov8_Project/_Weight_/2024-03-26'

# คัดลอกโฟลเดอร์ไปยังโฟลเดอร์ปลายทาง
shutil.copytree(source_path, destination_path)

In [ ]:
%pip install supervision

In [ ]:
from supervision.assets import download_assets, VideoAssets

download_assets(VideoAssets.PEOPLE_WALKING)